In [1]:
from data_loader import data_loader
from RNN_model import RNN_model

import torch.nn as nn

In [2]:
input_dim = 255 # Fake.

hidden_dim = 30
layer_dim = 2
output_dim = 4  # Four kinds of genres within 12 songs.


In [4]:



seqs, labels = data_loader('music-affect_v2')



1 :  (16, 1028)
2 :  (16, 980)
3 :  (16, 932)
4 :  (16, 964)
5 :  (16, 868)
6 :  (16, 836)
7 :  (16, 808)
8 :  (16, 840)
9 :  (16, 956)
10 :  (16, 1104)
11 :  (16, 956)
12 :  (16, 965)


In [5]:
print(seqs)
print(labels.shape)
    

[        1         2         3         4         5         6         7     \
1    4.57122   4.68903   4.43547   4.86448   4.92083   4.91187   4.82991   
2   0.307353  0.307353  0.279179  0.308634  0.304792  0.304792  0.308634   
3   0.098615  0.119106  0.105019  0.102457  0.101176  0.101176  0.099896   
4   0.090925  0.093486  0.093486  0.093486  0.092206  0.093486  0.090925   
5    1.28255   2.10324    1.3786   1.27723   1.46036   1.44499   1.39889   
6   0.058909  0.058909  0.057629  0.057629  0.057629  0.055067  0.055067   
7   0.430294  0.450784  0.444381   0.44182  0.445662  0.446942  0.446942   
8   0.138309  0.138309  0.137028  0.138309  0.138309  0.135748  0.138309   
9   0.046103  0.046103  0.046103  0.044822  0.044822  0.044822  0.046103   
10  0.107574  0.106293  0.106293  0.108854  0.106293  0.107574  0.107574   
11  0.129344  0.165202   0.16136  0.147273  0.142151  0.138309   0.14087   
12  0.559905  0.558625  0.558625  0.559905  0.562466  0.565028  0.576553   
13  0.17928

AttributeError: 'list' object has no attribute 'shape'

In [ ]:

lengths = 1104
rnn = RNN_model(input_dim, hidden_dim, layer_dim, output_dim)

x = nn.utils.rnn.pack_padded_sequence(data, lengths, batch_first=False)
x, h = rnn(x)
x, lengths = nn.utils.rnn.pad_packed_sequence(x, batch_first=False)



In [ ]:

print(rnn)


print(rnn.rnn.all_weights)
print("+++++++++++++++++++++++++++++++++++++")
print(rnn.fc.weight)

